# Todos



## Step
    
**Pws ypologizw to reward apo to `action`**
    
- [ ] Value of state from baselinenet
- [ ] Value of contract
- [ ] CII
- [ ] Lateness
        
**Pws na kanw swsta update to state**
        
- [ ] contract tensor
- [ ] ship tensor
- [ ] contract mask
- [ ] ship mask
- [ ] ship_log
        

## Train 

ti einai ta `advantages`, `returns` kai giati xrhsimopoioyntai:
        
- [ ] sthn baseline_net.update(observations, returns) gia to update to baselineNet
- [ ] sthn PolicyGradient.update_policy(observations, actions, advantages)gia to update ths policygradient 

# Code

In [1]:
# autoreloading modules that i change in vscode
%load_ext autoreload
%autoreload 2

In [2]:
import gym
import tensorflow as tf
import tensorflow_probability as tfp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys

In [3]:
# load data
data_dict = {
    "fleet_path": "data/fleet_small.csv",
    "ports_path": "data/ports_10.csv",
    "dm_path": "data/distance_matrix.csv",
}


ports_df = pd.read_csv(data_dict["ports_path"])
fleet_df = pd.read_csv(data_dict["fleet_path"])
distance_matrix = pd.read_csv(data_dict["dm_path"])

In [4]:
from env.env import CarbonEnv
from models.models import BaselineNet, PolicyNet, CarbonModel

In [ ]:
# run this cell when needing to reload the classes CarbonEnv, BaselineNet, PolicyNet

del sys.modules['models.models']
del sys.modules['env.env']

from env.env import CarbonEnv
from models.models import BaselineNet, PolicyNet, CarbonModel

In [5]:
envo = CarbonEnv()

2021-11-09 15:01:52.071531: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-11-09 15:01:52.072695: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [6]:
envo.action_space_dim[0]

13

In [30]:
envo.state

{'contracts_state': <tf.Tensor: shape=(4, 10), dtype=float32, numpy=
 array([[9.0000e+00, 3.0000e+00, 1.0000e+00, 0.0000e+00, 8.0000e+00,
         4.6560e+04, 9.0000e+00, 1.0000e+00, 1.1470e+03, 6.0000e+00],
        [7.0000e+00, 5.0000e+00, 3.0000e+00, 0.0000e+00, 8.0000e+00,
         6.1445e+04, 9.0000e+00, 1.0000e+00, 1.2230e+03, 8.0000e+00],
        [2.0000e+00, 6.0000e+00, 2.0000e+00, 0.0000e+00, 1.3000e+01,
         5.9396e+04, 1.4000e+01, 1.0000e+00, 2.4630e+03, 1.0000e+01],
        [2.0000e+00, 7.0000e+00, 3.0000e+00, 0.0000e+00, 1.3000e+01,
         6.3993e+04, 1.4000e+01, 1.0000e+00, 2.5030e+03, 1.1000e+01]],
       dtype=float32)>,
 'ships_state': <tf.Tensor: shape=(4, 11), dtype=float32, numpy=
 array([[1.0000000e+00, 5.0000000e+04, 5.3851833e+00, 0.0000000e+00,
         9.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
         6.8800000e+02, 2.7640000e+03, 2.7640000e+03],
        [2.0000000e+00, 6.0000000e+04, 4.8078370e+00, 0.0000000e+00,
         8.0000000e+00,

In [ ]:
# something
s = np.array([[0], [0], [0], [1]])


k = np.where(s==0,3*[False],3*[True])

k

k.ravel()

st = tf.convert_to_tensor(s)
st

bm = tf.equal(st,0)


skou = tf.where(bm,tf.repeat(tf.constant(False),3),tf.repeat(tf.constant(True),3))
skou

skou = tf.reshape(skou,[-1])
skou

skou = tf.expand_dims(skou,axis=1)
skou

bol = tf.constant(False,shape=(1,1))

tf.concat((skou,bol),axis=0)

In [8]:
initial_state = envo.state

In [9]:
baseline_net = BaselineNet(embedding_size=envo.embedding_size)

In [10]:
policy_net = PolicyNet(embedding_size=envo.embedding_size,output_size=envo.action_space_dim[0])

In [11]:
action = policy_net.sample_action(initial_state)

In [12]:
action

4

In [26]:
envo.step(action,ship=1)

The contract is contract_1 and the speed for ship 1 is 12 knots


In [27]:
envo.step(0,2)

The contract is contract_0 and the speed for ship 2 is 10 knots


In [20]:
envo.ships_tensor[:,0]

<tf.Tensor: shape=(4,), dtype=float32, numpy=array([1., 2., 3., 4.], dtype=float32)>

In [31]:
envo.ships_tensor

[autoreload of utils.utils failed: Traceback (most recent call last):
  File "/home/pristakos/anaconda3/envs/rl/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/pristakos/anaconda3/envs/rl/lib/python3.9/site-packages/IPython/extensions/autoreload.py", line 394, in superreload
    module = reload(module)
  File "/home/pristakos/anaconda3/envs/rl/lib/python3.9/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/home/pristakos/anaconda3/envs/rl/lib/python3.9/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 613, in _exec
  File "<frozen importlib._bootstrap_external>", line 846, in exec_module
  File "<frozen importlib._bootstrap_external>", line 983, in get_code
  File "<frozen importlib._bootstrap_external>", line 913, in source_to_code
  File "<frozen importlib._bootstrap>", line 228, in _call_with_fr

<tf.Tensor: shape=(4, 11), dtype=float32, numpy=
array([[1.0000000e+00, 5.0000000e+04, 5.3851833e+00, 0.0000000e+00,
        9.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
        6.8800000e+02, 2.7640000e+03, 2.7640000e+03],
       [2.0000000e+00, 6.0000000e+04, 4.8078370e+00, 0.0000000e+00,
        8.0000000e+00, 0.0000000e+00, 1.0000000e+00, 2.8700000e+02,
        1.1400000e+03, 1.4600000e+03, 1.4600000e+03],
       [3.0000000e+00, 7.0000000e+04, 4.3682637e+00, 0.0000000e+00,
        6.0000000e+00, 0.0000000e+00, 1.0000000e+00, 4.1200000e+02,
        5.0200000e+02, 2.4630000e+03, 2.4630000e+03],
       [4.0000000e+00, 8.0000000e+04, 4.0201092e+00, 0.0000000e+00,
        1.0000000e+01, 0.0000000e+00, 1.0000000e+00, 1.1740000e+03,
        9.0800000e+02, 1.6210000e+03, 1.6210000e+03]], dtype=float32)>

In [42]:
0 % 4 + 7

0

In [43]:
1 % 4 + 7

1

In [44]:
2 % 4

2

In [45]:
3 % 4

3

In [29]:
envo.ships_tensor[0,:]

<tf.Tensor: shape=(11,), dtype=float32, numpy=
array([1.0000000e+00, 5.0000000e+04, 5.3851833e+00, 0.0000000e+00,
       9.0000000e+00, 0.0000000e+00, 1.0000000e+00, 0.0000000e+00,
       6.8800000e+02, 2.7640000e+03, 2.7640000e+03], dtype=float32)>

In [ ]:
for i in range(0, 8):

    day = i // 4
    print(day)

In [ ]:
# encoder mlp

# see https://keras.io/examples/structured_data/structured_data_classification_from_scratch/
# see https://www.tensorflow.org/tutorials/generative/autoencoder
# Create an Integer Categorical Feature for contract_type

In [ ]:
# Delay

contracts = create_contrs(seed=56)
agent_speed = 10
agent_ship = 2
ship_current_port = 4
contract = contracts.loc[0]
start_port_number = contract.start_port_number
end_port_number = contract.end_port_number
trip_1_hours = find_distance(ship_current_port,start_port_number) / agent_speed
print(dt_1_hours)
trip_2_hours = find_distance(start_port_number,end_port_number) / agent_speed
print(dt_2_hours)
trip_total_hours = trip_1_hours + trip_2_hours


trip_total_days = round(trip_total_hours / 24)
print(trip_total_days)

contract_duration = contract.contract_duration
delay = trip_total_days - contract_duration
delay_penalty_factor_per_day = 10

reward = 0
penalty = 0 
if delay <= 0:
    # I arrived on time
    # provide a constant reward > 0 
    reward = 10
    # 
else:
    # I arrived late 
    # penalty < 0
    delay_in_days = -delay
    penalty = delay_penalty_factor_per_day * delay_in_days

print(f"The delay is {delay} days")
print(f"The reward is {reward}")
print(f"The penalty is {penalty}")

# cargo_size vs ship_shize

contracts = create_contrs(seed=56)

# gia ka8e contract se spawned contracts

contract = contracts.loc[0]

fleet_df
# elegxw ka8e ploio pou einai available dhladh exei availability 1 
mask_of_available_ships = fleet_df['ship_availability'] == 1

available_ships = fleet_df[mask_of_available_ships]

available_ships_numbers = available_ships.ship_number

available_ships_numbers



# checkare sto masking an kaneis mask out tous assous h ta mhdenika

# gia ka8e tetoio ploio


# get ship_number

ship_numbers = 
agent_ship_selection = 2

# get ship info
ship_mask = fleet_df['ship_number'] == agent_ship_selection

ship_info = fleet_df[ship_mask]
print(ship_info)

ship_current_port = ship_info.current_port
print(ship_current_port)






start_port_number = contract.start_port_number
end_port_number = contract.end_port_number
trip_1_distance = find_distance(ship_current_port,start_port_number)
print(trip_1_distance)
trip_2_distance = find_distance(start_port_number,end_port_number)
print(trip_2_distance)
trip_total_distance = trip_1_distance + trip_2_distance




trip_total_days = round(trip_total_hours / 24)
print(trip_total_days)

contract_duration = contract.contract_duration
delay = trip_total_days - contract_duration
delay_penalty_factor_per_day = 10

reward = 0
penalty = 0 
if delay <= 0:
    # I arrived on time
    # provide a constant reward > 0 
    reward = 10
    # 
else:
    # I arrived late 
    # penalty < 0
    delay_in_days = -delay
    penalty = delay_penalty_factor_per_day * delay_in_days

print(f"The delay is {delay} days")
print(f"The reward is {reward}")
print(f"The penalty is {penalty}")
